# Benchmarking Our Algorithm against others

In [5]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [7]:
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 100

# Specify the Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [8]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
max_N = 40
N_range = np.arange(1,max_N+1)

In [14]:
# results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Env: MultiTunnel Alg: Seq_Tube N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 9/11
Planning for agent 10/11
On time for algorithm Seq_Tube N agent =  11
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 9/11
Planning for agent 10/11
On time for algorithm Seq_Tube_nonLazy N agent =  11
Env: MultiTunnel Alg: Seq_S2M2 N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 